In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import matthews_corrcoef
from scipy.stats import mode
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

Load baseline ensemble version & Autogluon result

In [ ]:
# Load probabilities from CSV files
# Result from "only competition dataset"
# (Try to use original dataset too if it has better performance)
# Auto 의 경우 prob 계산을 우리 형태에 맞게 다시 해줘야 될듯
auto_train_df = pd.read_csv('pred/submission_autogluon_train_proba.csv')
base_train_df = pd.read_csv('pred/submission_stack_base_train_proba.csv')
auto_train_probs = auto_train_df['class']
base_train_probs = base_train_df['class']

auto_test_df = pd.read_csv('pred/submission_autogluon_test_proba.csv')
base_test_df = pd.read_csv('pred/submission_stack_base_test_proba.csv')
auto_test_probs = auto_test_df['class']
base_test_probs = base_test_df['class']

train_df = pd.read_csv("data/playground-series-s4e8/train.csv", index_col='id')
train_y = train_df['class']
lab_enc = LabelEncoder().fit(train_y)
train_y = lab_enc.transform(train_y)

In [ ]:
print(auto_train_probs.shape)
print(base_train_probs.shape)

print(auto_test_probs.shape)
print(base_test_probs.shape)

Define Ensemble Method

In [ ]:
#voting hard & soft
#voting hard & soft
def voting_ensemble(oof_probs, y, threshold=0.5, voting_type='soft'):
    if voting_type == 'soft':
        ensemble_preds = oof_probs.mean(axis=1)
        ensemble_class_preds = (ensemble_preds > threshold).astype(int)
        
    elif voting_type == 'hard':
        binary_preds = (oof_probs > threshold).astype(int)
        ensemble_class_preds = mode(binary_preds, axis=1)[0].flatten()
    
    mcc_score = matthews_corrcoef(y, ensemble_class_preds)
    
    return mcc_score, ensemble_class_preds

def voting_ensemble_only_pred(oof_probs, threshold=0.5, voting_type='soft'):
  if voting_type == 'soft':
      ensemble_preds = oof_probs.mean(axis=1)
      ensemble_class_preds = (ensemble_preds > threshold).astype(int)
      
  elif voting_type == 'hard':
      binary_preds = (oof_probs > threshold).astype(int)
      ensemble_class_preds = mode(binary_preds, axis=1)[0].flatten()

  return ensemble_class_preds
train_df = pd.read_csv("data/playground-series-s4e8/train.csv", index_col='id')

random_state = 101
meta_model = LogisticRegression(**metatest_probs_model_params, random_state=random_state)

Conduct Ensemble

1. Voting

In [ ]:
train_probs, test_probs = pd.DataFrame(), pd.DataFrame()

train_probs['autogluon'] = auto_train_probs
train_probs['base_ensemble'] = base_train_probs

test_probs['autogluon'] = auto_test_probs
test_probs['base_ensemble'] = base_test_probs

In [ ]:
# Get CV score
train_soft_vote_score, train_soft_vote_pred = voting_ensemble(train_probs, train_y, voting_type='soft')
train_hard_vote_score, train_hard_vote_pred = voting_ensemble(train_probs, train_y, voting_type='hard')

print(f"Soft Voting MCC (CV score): {train_soft_vote_score}")
print(f"Hard Voting MCC (CV score): {train_hard_vote_score}")

In [ ]:
# Get Test predictions
test_soft_vote_pred = voting_ensemble_only_pred(test_probs, voting_type='soft')
test_hard_vote_pred = voting_ensemble_only_pred(test_probs, voting_type='hard')

2. Stacking

In [ ]:
# Train Meta Model
meta_model = meta_model.fit(train_probs, train_y)

In [ ]:
# Get CV score
train_stack_pred = meta_model.predict(train_probs)
train_stack_score = matthews_corrcoef(train_y, train_stack_pred)

print(f"Tacking MCC (CV score): {train_stack_score}")

In [ ]:
# Get Test predictions
test_stack_pred = meta_model.predict(test_probs)

Submission

1. Soft Vote

In [ ]:
''' Soft Voting '''
test_soft_vote_pred = lab_enc.inverse_transform(test_soft_vote_pred)
sub = pd.read_csv('data/playground-series-s4e8/sample_submission.csv')
sub['class'] = test_soft_vote_pred
sub.to_csv('pred/sub_auto_base_soft_vote.csv', index=False)

2. Hard Vote

In [ ]:
''' Hard Voting '''
test_hard_vote_pred = lab_enc.inverse_transform(test_hard_vote_pred)
sub = pd.read_csv('data/playground-series-s4e8/sample_submission.csv')
sub['class'] = test_hard_vote_pred
sub.to_csv('pred/sub_auto_base_hard_vote.csv', index=False)

3. Stacking

In [ ]:
''' Stacking '''
test_stack_pred = lab_enc.inverse_transform(test_stack_pred)
sub = pd.read_csv('data/playground-series-s4e8/sample_submission.csv')
sub['class'] = test_stack_pred
sub.to_csv('pred/sub_auto_base_stack.csv', index=False)